In [1]:
import pyspark
from pyspark.sql import SparkSession, types
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, IntegerType, DateType, StructField, StringType, TimestampType
import pyspark.pandas as ps
import logging, traceback
import requests
import sys
import pandas as pd
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
from dateutil.parser import parse

/home/camiloms/spark/spark-3.3.3-bin-hadoop3/python/pyspark/pandas/__init__.py:49: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [ ]:
# This is to create a Local Cluster 
#spark = SparkSession.builder \
#        .master("spark://LAPTOP-C9HBU13M.:7077") \
#        .appName('test') \
#        .getOrCreate()

In [ ]:
#Spark version testing for Local Cluster

#spark

In [2]:
pyspark.__file__

'/home/camiloms/spark/spark-3.3.3-bin-hadoop3/python/pyspark/__init__.py'

In [3]:
credentials_json ='/home/camiloms/nba_stats_de/airflow/.google/credentials/google_credentials.json'

conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('test') \
    .set("spark.jars", "./lib/gcs-connector-hadoop3-2.2.5.jar") \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile",credentials_json)

In [4]:
sc = SparkContext(conf=conf)

sc._jsc.hadoopConfiguration().set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
sc._jsc.hadoopConfiguration().set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
sc._jsc.hadoopConfiguration().set("fs.gs.auth.service.account.json.keyfile", credentials_json)
sc._jsc.hadoopConfiguration().set("fs.gs.auth.service.account.enable", "true")

23/09/07 13:51:44 WARN Utils: Your hostname, LAPTOP-C9HBU13M resolves to a loopback address: 127.0.1.1; using 172.21.82.95 instead (on interface eth0)
23/09/07 13:51:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/09/07 13:51:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/09/07 13:51:48 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()

In [12]:
df_current = spark.read.parquet('gs://import-bucket-camiloms10/raw/players_game_stats_current_year.parquet')

In [13]:
df_current.show()

+-----------+---------+--------------------+------------+----------+-----------------+--------------------+----------+-------------------+-----------+---+------------------+---+---+------+----+----+-------+---+---+------+----+----+---+---+---+---+---+----+---+---+---+----------+---------------+---+---+----------------+-------+------+------+----------+--------+--------+--------+-----------+---------+---------+------------+--------+--------+-----------+---------+---------+--------+--------+--------+--------+--------+---------+-------+--------+--------+---------------+--------------------+--------+--------+---------------------+--------------+-----------------+
|SEASON_YEAR|PLAYER_ID|         PLAYER_NAME|    NICKNAME|   TEAM_ID|TEAM_ABBREVIATION|           TEAM_NAME|   GAME_ID|          GAME_DATE|    MATCHUP| WL|               MIN|FGM|FGA|FG_PCT|FG3M|FG3A|FG3_PCT|FTM|FTA|FT_PCT|OREB|DREB|REB|AST|TOV|STL|BLK|BLKA| PF|PFD|PTS|PLUS_MINUS|NBA_FANTASY_PTS|DD2|TD3|WNBA_FANTASY_PTS|GP_RANK|W_RAN

In [14]:
df_past = spark.read.parquet('gs://import-bucket-camiloms10/raw/players_game_stats_till_last_year.parquet')

In [15]:
df_past.show()

+---------+---------+------------------+--------+----------+-----------------+-----------------+----------+-------------------+-----------+---+---+---+----+------+----+----+-------+---+---+------+----+----+---+---+----+----+----+---+---+----------+---------------+-----------+----+----+---------------+----+----+----------------+-------+------+------+----------+--------+--------+--------+-----------+---------+---------+------------+--------+--------+-----------+---------+---------+--------+--------+--------+--------+--------+---------+-------+--------+--------+---------------+--------------------+--------+--------+---------------------+--------------+-----------------+
|SEASON_ID|PLAYER_ID|       PLAYER_NAME|NICKNAME|   TEAM_ID|TEAM_ABBREVIATION|        TEAM_NAME|   GAME_ID|          GAME_DATE|    MATCHUP| WL|MIN|FGM| FGA|FG_PCT|FG3M|FG3A|FG3_PCT|FTM|FTA|FT_PCT|OREB|DREB|REB|AST| STL| BLK| TOV| PF|PTS|PLUS_MINUS|VIDEO_AVAILABLE|SEASON_YEAR|BLKA| PFD|NBA_FANTASY_PTS| DD2| TD3|WNBA_FANTASY

In [16]:
full_dataset = df_past.unionByName(df_current, allowMissingColumns=True)

In [17]:
full_dataset.show()

+---------+---------+------------------+--------+----------+-----------------+-----------------+----------+-------------------+-----------+---+---+---+----+------+----+----+-------+---+---+------+----+----+---+---+----+----+----+---+---+----------+---------------+-----------+----+----+---------------+----+----+----------------+-------+------+------+----------+--------+--------+--------+-----------+---------+---------+------------+--------+--------+-----------+---------+---------+--------+--------+--------+--------+--------+---------+-------+--------+--------+---------------+--------------------+--------+--------+---------------------+--------------+-----------------+
|SEASON_ID|PLAYER_ID|       PLAYER_NAME|NICKNAME|   TEAM_ID|TEAM_ABBREVIATION|        TEAM_NAME|   GAME_ID|          GAME_DATE|    MATCHUP| WL|MIN|FGM| FGA|FG_PCT|FG3M|FG3A|FG3_PCT|FTM|FTA|FT_PCT|OREB|DREB|REB|AST| STL| BLK| TOV| PF|PTS|PLUS_MINUS|VIDEO_AVAILABLE|SEASON_YEAR|BLKA| PFD|NBA_FANTASY_PTS| DD2| TD3|WNBA_FANTASY

In [18]:
full_dataset.fillna(0)

DataFrame[SEASON_ID: string, PLAYER_ID: bigint, PLAYER_NAME: string, NICKNAME: string, TEAM_ID: bigint, TEAM_ABBREVIATION: string, TEAM_NAME: string, GAME_ID: string, GAME_DATE: timestamp, MATCHUP: string, WL: string, MIN: double, FGM: bigint, FGA: double, FG_PCT: double, FG3M: double, FG3A: double, FG3_PCT: double, FTM: bigint, FTA: bigint, FT_PCT: double, OREB: double, DREB: double, REB: bigint, AST: bigint, STL: double, BLK: double, TOV: double, PF: bigint, PTS: bigint, PLUS_MINUS: bigint, VIDEO_AVAILABLE: double, SEASON_YEAR: string, BLKA: double, PFD: double, NBA_FANTASY_PTS: double, DD2: double, TD3: double, WNBA_FANTASY_PTS: double, GP_RANK: double, W_RANK: double, L_RANK: double, W_PCT_RANK: double, MIN_RANK: double, FGM_RANK: double, FGA_RANK: double, FG_PCT_RANK: double, FG3M_RANK: double, FG3A_RANK: double, FG3_PCT_RANK: double, FTM_RANK: double, FTA_RANK: double, FT_PCT_RANK: double, OREB_RANK: double, DREB_RANK: double, REB_RANK: double, AST_RANK: double, TOV_RANK: double,

In [27]:
full_dataset.createOrReplaceTempView("table_df")
spark.sql("""SELECT * FROM table_df ORDER BY GAME_DATE DESC limit 1""").show()

+---------+---------+-----------+--------+----------+-----------------+------------------+----------+-------------------+---------+---+-----+---+----+------+----+----+-------+---+---+------+----+----+---+---+---+---+---+---+---+----------+---------------+-----------+----+---+---------------+---+---+----------------+-------+------+------+----------+--------+--------+--------+-----------+---------+---------+------------+--------+--------+-----------+---------+---------+--------+--------+--------+--------+--------+---------+-------+--------+--------+---------------+--------------------+--------+--------+---------------------+--------------+-----------------+
|SEASON_ID|PLAYER_ID|PLAYER_NAME|NICKNAME|   TEAM_ID|TEAM_ABBREVIATION|         TEAM_NAME|   GAME_ID|          GAME_DATE|  MATCHUP| WL|  MIN|FGM| FGA|FG_PCT|FG3M|FG3A|FG3_PCT|FTM|FTA|FT_PCT|OREB|DREB|REB|AST|STL|BLK|TOV| PF|PTS|PLUS_MINUS|VIDEO_AVAILABLE|SEASON_YEAR|BLKA|PFD|NBA_FANTASY_PTS|DD2|TD3|WNBA_FANTASY_PTS|GP_RANK|W_RANK|L_RA

In [ ]:
schema = types.StructType([
                         types.StructField("SEASON_ID", types.StringType(), True),
                         types.StructField("PLAYER_ID", types.IntegerType(), True),
                         types.StructField("PLAYER_NAME", types.StringType(), True),
                         types.StructField("NICKNAME", types. StringType(), True),
                         types.StructField("TEAM_ID", types.IntegerType(), True),
                         types.StructField("TEAM_ABBREVIATION", types.StringType(), True),
                         types.StructField("TEAM_NAME", types. StringType(), True),
                         types.StructField("GAME_ID", types.StringType(), True),
                         types.StructField("GAME_DATE", types.StringType(), True),
                         types.StructField("MATCHUP", types.StringType(), True),
                         types.StructField("WL", types.StringType(), True),
                         types.StructField("MIN", types.IntegerType(), True),
                         types.StructField("FGM", types.IntegerType(), True),
                         types.StructField("FGA", types.FloatType(), True),
                         types.StructField("FG_PCT", types.FloatType(), True),
                         types.StructField("FG3M", types.IntegerType(), True),
                         types.StructField("FG3A", types.IntegerType(), True),
                         types.StructField("FG3_PCT", types.FloatType(), True),
                         types.StructField("FTM", types.IntegerType(), True),
                         types.StructField("FTA", types. IntegerType(), True),
                         types.StructField("FT_PCT", types.FloatType(), True),
                         types.StructField("OREB", types.IntegerType(), True),
                         types.StructField("DREB", types.IntegerType(), True),
                         types.StructField("REB", types.IntegerType(), True),
                         types.StructField("AST", types.IntegerType(), True),
                         types.StructField("STL", types.IntegerType(), True),
                         types.StructField("BLK", types.IntegerType(), True),
                         types.StructField("TOV", types.IntegerType(), True),
                         types.StructField("PF", types.IntegerType(), True),
                         types.StructField("PTS", types.IntegerType(), True),
                         types.StructField("PLUS_MINUS", types.IntegerType(), True),
                         types.StructField("VIDEO_AVAILABLE", types.IntegerType(), True)
])